In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%store -r dm 


dm = dm

In [4]:
dm.combine_all_data()

['X_test_estimated_a', 'data_A']
['X_test_estimated_b', 'data_B']
['X_test_estimated_c', 'data_C']


In [5]:
from sklearn.model_selection import train_test_split 

X = dm.data.iloc[:, 2:]
y = dm.data.iloc[:, 1]

X_train, X_test, y_train, y_test = train_test_split(X, y)

X_submission = dm.X_test_estimated[dm.X_test_estimated.columns.intersection(X.columns)]

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape, X_submission.shape)

X

(36588, 48) (36588,) (12196, 48) (12196,) (2160, 48)


,absolute_humidity_2m:gm3,air_density_2m:kgm3,clear_sky_energy_1h:J,clear_sky_rad:W,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,diffuse_rad_1h:J,direct_rad:W,direct_rad_1h:J,...,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,month,hours,week_number,day_year,sum_rad:W,location
0,7.700,1.22825,0.000000,0.000,0.0,280.299988,0.000,0.000000,0.00,0.000000,...,3.600,-3.575,-0.500,0.0,12.0,1.607695,49.021857,341.969796,0.000000,0
1,7.700,1.22350,0.000000,0.000,0.0,280.299988,0.000,0.000000,0.00,0.000000,...,3.350,-3.350,0.275,0.0,12.0,0.408890,49.021857,341.969796,0.000000,0
2,7.875,1.21975,0.000000,0.000,0.0,280.649994,0.000,0.000000,0.00,0.000000,...,3.050,-2.950,0.750,0.0,12.0,0.000000,50.310422,343.473765,0.000000,0
3,8.425,1.21800,208.649994,0.750,0.0,281.674988,0.300,526.775024,0.00,0.000000,...,2.725,-2.600,0.875,0.0,12.0,0.408890,50.310422,343.473765,0.350000,0
4,8.950,1.21800,32468.150391,23.100,0.0,282.500000,11.975,22068.949219,0.15,282.975006,...,2.550,-2.350,0.925,0.0,12.0,1.607695,50.310422,343.473765,11.741668,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48779,4.400,1.27550,84010.148438,4.175,0.0,272.024994,2.775,54774.000000,0.00,9934.575195,...,4.075,3.600,1.875,0.0,9.0,8.894171,38.082802,269.178571,2.316667,2
48780,4.400,1.27850,2206.800049,0.000,0.0,271.950012,0.000,4984.049805,0.00,0.000000,...,3.600,2.950,2.125,0.0,9.0,6.000000,38.082802,269.178571,0.000000,2
48781,4.400,1.27900,0.000000,0.000,0.0,271.899994,0.000,0.000000,0.00,0.000000,...,3.600,2.625,2.400,0.0,9.0,3.514719,38.082802,269.178571,0.000000,2
48782,4.400,1.27975,0.000000,0.000,0.0,271.950012,0.000,0.000000,0.00,0.000000,...,3.275,2.325,2.325,0.0,9.0,1.607695,38.082802,269.178571,0.000000,2


In [8]:
import optuna
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 5, 100),
        'criterion': trial.suggest_categorical('criterion', ["squared_error", "absolute_error", "poisson", "MAE"]),
    }
    model = RandomForestRegressor(**param)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return mean_absolute_error(y_test, y_pred)

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50, timeout=600)

[I 2023-11-05 14:52:01,443] A new study created in memory with name: no-name-9192c53b-066b-4702-9222-6e49ff1591d3


[I 2023-11-05 18:48:07,599] Trial 0 finished with value: 153.7147593898097 and parameters: {'n_estimators': 255, 'max_depth': 73, 'criterion': 'absolute_error'}. Best is trial 0 with value: 153.7147593898097.
